# Source Code | ソースコードこのノートブックでは、言語解析を用いた特別なアプローチでソースコードファイルを読み込む方法について説明しています：コード内の各トップレベル関数とクラスは、別々のドキュメントに読み込まれます。既に読み込まれた関数やクラスの外にあるその他のトップレベルコードも、別のドキュメントとして読み込まれます。> This notebook covers how to load source code files using a special approach with language parsing: each top-level function and class in the code is loaded into separate documents. Any remaining code top-level code outside the already loaded functions and classes will be loaded into a separate document.このアプローチは、ソースコード上のQAモデルの精度を向上させる可能性があります。現在、コード解析に対応している言語はPythonとJavaScriptです。解析に使用する言語は設定可能であり、構文に基づいて分割を行うために必要な最小行数も設定できます。> This approach can potentially improve the accuracy of QA models over source code. Currently, the supported languages for code parsing are Python and JavaScript. The language used for parsing can be configured, along with the minimum number of lines required to activate the splitting based on syntax.

In [ ]:
! pip install esprima

In [1]:
import warnings

warnings.filterwarnings("ignore")
from pprint import pprint

from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import LanguageParser
from langchain.text_splitter import Language

In [2]:
loader = GenericLoader.from_filesystem(
    "./example_data/source_code",
    glob="*",
    suffixes=[".py", ".js"],
    parser=LanguageParser(),
)
docs = loader.load()

In [3]:
len(docs)

6

In [4]:
for document in docs:
    pprint(document.metadata)

{'content_type': 'functions_classes',
 'language': <Language.PYTHON: 'python'>,
 'source': 'example_data/source_code/example.py'}
{'content_type': 'functions_classes',
 'language': <Language.PYTHON: 'python'>,
 'source': 'example_data/source_code/example.py'}
{'content_type': 'simplified_code',
 'language': <Language.PYTHON: 'python'>,
 'source': 'example_data/source_code/example.py'}
{'content_type': 'functions_classes',
 'language': <Language.JS: 'js'>,
 'source': 'example_data/source_code/example.js'}
{'content_type': 'functions_classes',
 'language': <Language.JS: 'js'>,
 'source': 'example_data/source_code/example.js'}
{'content_type': 'simplified_code',
 'language': <Language.JS: 'js'>,
 'source': 'example_data/source_code/example.js'}


In [7]:
print("\n\n--8<--\n\n".join([document.page_content for document in docs]))

class MyClass:
    def __init__(self, name):
        self.name = name

    def greet(self):
        print(f"Hello, {self.name}!")

--8<--

def main():
    name = input("Enter your name: ")
    obj = MyClass(name)
    obj.greet()

--8<--

# Code for: class MyClass:


# Code for: def main():


if __name__ == "__main__":
    main()

--8<--

class MyClass {
  constructor(name) {
    this.name = name;
  }

  greet() {
    console.log(`Hello, ${this.name}!`);
  }
}

--8<--

function main() {
  const name = prompt("Enter your name:");
  const obj = new MyClass(name);
  obj.greet();
}

--8<--

// Code for: class MyClass {

// Code for: function main() {

main();


パーサーは、小さなファイルの場合無効にすることができます。> The parser can be disabled for small files.`parser_threshold` パラメーターは、パーサーを使用してセグメント化する際にソースコードファイルが最低限持っていなければならない行数を示しています。> The parameter `parser_threshold` indicates the minimum number of lines that the source code file must have to be segmented using the parser.

In [8]:
loader = GenericLoader.from_filesystem(
    "./example_data/source_code",
    glob="*",
    suffixes=[".py"],
    parser=LanguageParser(language=Language.PYTHON, parser_threshold=1000),
)
docs = loader.load()

In [9]:
len(docs)

1

In [10]:
print(docs[0].page_content)

class MyClass:
    def __init__(self, name):
        self.name = name

    def greet(self):
        print(f"Hello, {self.name}!")


def main():
    name = input("Enter your name: ")
    obj = MyClass(name)
    obj.greet()


if __name__ == "__main__":
    main()



## Splitting | 分割大きすぎる関数、クラス、またはスクリプトについては、追加の分割が必要になる場合があります。> Additional splitting could be needed for those functions, classes, or scripts that are too big.

In [11]:
loader = GenericLoader.from_filesystem(
    "./example_data/source_code",
    glob="*",
    suffixes=[".js"],
    parser=LanguageParser(language=Language.JS),
)
docs = loader.load()

In [12]:
from langchain.text_splitter import (
    Language,
    RecursiveCharacterTextSplitter,
)

In [13]:
js_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.JS, chunk_size=60, chunk_overlap=0
)

In [14]:
result = js_splitter.split_documents(docs)

In [15]:
len(result)

7

In [16]:
print("\n\n--8<--\n\n".join([document.page_content for document in result]))

class MyClass {
  constructor(name) {
    this.name = name;

--8<--

}

--8<--

greet() {
    console.log(`Hello, ${this.name}!`);
  }
}

--8<--

function main() {
  const name = prompt("Enter your name:");

--8<--

const obj = new MyClass(name);
  obj.greet();
}

--8<--

// Code for: class MyClass {

// Code for: function main() {

--8<--

main();
